# Accesibilidad rural en Asturias

Este documento explica cómo calcular la accesibilidad rural a hospitales, y otros proveedores de servicios básicos, como bancos o supermercados.

Permite responder a preguntas como:
* ¿Qué porcentaje de asturianos puede no llega, conduciendo, a su hospital más cercano en 1 hora?
* ¿Cómo favorece la mejora de cierta carretera en el acceso más rápido a servicios básicos?
* ¿Qué pasa si cierta carretera queda bloqueada?

Para responder a estas preguntas, sólo hacer falta usar software y datos libre, en concreto:
* [Open Street Map](http://osm.org). Mapa de datos libres, de donde sacamos las carreteras. Es posible conseguir los datos de carreteras oficiales del [Instituto Geográfico Nacional](http://www.ign.es/web/ign/portal), pero el programa que usaremos para obtener los tiempos de conducción usa una nomenclatura de tipos de carretera diferente.
* [Open Street Routing Map](http://osrm.org). Programa de enrutado. Utilizando como base los datos y nomenclatura de OSM es capaz de calcular la ruta más rápida según el tipo de transporte (andando, coche, bici, ...) y la velocidad típica de cada tipo de carretera (autopista, nacional, pavimentada, caleya, ...)
* [Python](). Usamos el lenguaje de programación python, y una serie de librerías dentro de ésta, para calcular todos los datos de accesibilidad.

## Obtener datos OSM

Este paso descarga todas las carreteras de Asturias, y una abudante franja alrededor (para incluir casos donde la mejor ruta para llegar desde algún punto de Asturias a otro incluye salir temporalmente a otra provincia.

### Instalar software

Además de python y jupyter para poder correr este documento, hacen falta un par de utilidades:
* osmconvert:
 `wget -O - http://m.m.i24.cc/osmconvert.c | cc -x c - -lz -O3 -o osmconvert`
* osmupdate
`wget -O - http://m.m.i24.cc/osmupdate.c | cc -x c - -o osmupdate`
* [Gdal](http://www.gdal.org/)
* [docker](https://docs.docker.com/install/)

### 1. Descargar los datos de Asturias


Lo más facil es hacer un rectángulo que abarque todo asturias y un poco de Galicia, León y Cantabria. Lo suficiente para incluir las autopistas que quizás sea mejor usar en algunos casos cuando vamos de un sitio a otro de Asturias, e incluir los hospitales que puedan estar más cerca de la gente que vive cerca de la frontera.


[![](asturias-roi.png)](https://gist.github.com/brunosan/d6be05f3a58136fe7c0e816c684235f0)

En `bash` se puede descargar todo el OSM de esta región facilmente:

```sh
    wget https://gist.githubusercontent.com/brunosan/d6be05f3a58136fe7c0e816c684235f0/raw/03386f6f8734320f2bca8d46a4386109afcd0488/map.geojson
   ogrinfo map.geojson map | grep Extent
   BBOX="-7.822265625000001,42.69858589169842,-3.8671874999999996,43.6599240747891"

   #Download the latest OSM data:
   DATE=`date '+%Y-%m-%d'`
   mkdir data/
   cd data/
   wget -P data/ -O asturias_$DATE.osm "http://overpass.osm.rambler.ru/cgi/xapi_meta?*[bbox=$BBOX
```

**Opcionalmente** Si espacio en disco es un problema, podemos convertir el archivo `.osm` al formato `.o5m`, que en este caso reduce el tamaño de ~1 Gb a ~100 Mb. 

```sh
../osmconvert -v asturias_$DATE.osm -o=asturias_$DATE.o5m
```

**Actualizar**. OSM se actualiza constantemente. Si queremos actualizar a la última versión, en vez de descargar todos los datos de nuevo, podemos usar este comando para obtener sólo los cambios. Mucho más rápido.

```sh
../osmupdate -v asturias_$DATE.o5m asturias_$DATE_tmp.o5m -b=$BBOX
```

### 2. Correr el enrutador

Para obtener el tiempo de viaje, en coche por ejemplo, entre cualquier par de puntos, necesitamos saber cual es la ruta más rápida. Para esto usamos el enrutador [OSRM](https://github.com/Project-OSRM/osrm-backend), con un perfil de coche.

La elección del tipo de transporte es importante porque especifica qué tipo de carreteras puede usar (por ejemplo elimina los caminos peatonales, las vías de tren, ... Además, asigna velocidades típicas en función del tipo de vía o tipo de asfalto que figure en las propiedades del archivo de OSM. Por esto llevaría mucho más tiempo usar los archivos de vías oficiales, ya que el diccionario de tipos de vías y pavimento no es el mismo que OSM; y OSRM está diseñado para usar OSM.

Con `docker` instalado, instalar y correr el enrutador son 3 líneas:

```sh
#transformar los datos de OSM, con el perfil de coche
docker run -t -v $(pwd):/data osrm/osrm-backend osrm-extract -p /opt/car.lua /data/asturias_2018-03-30.o5m
docker run -t -v $(pwd):/data osrm/osrm-backend osrm-partition /data/asturias_2018-03-30.o5m.osrm
docker run -t -v $(pwd):/data osrm/osrm-backend osrm-customize /data/asturias_2018-03-30.o5m.osrm

#Correr el servicio de rutas localmente
docker run -t -i -p 5000:5000 -v $(pwd):/data osrm/osrm-backend osrm-routed --algorithm mld /data/asturias_2018-03-30.o5m.osrm
```

Aunque el servicio ya está funcionando, también es posible usar un visualizador usual, que en este caso, funciona localmente en [http://127.0.0.1:9966](http://127.0.0.1:9966)
```sh
docker run -p 9966:9966 osrm/osrm-frontend
```

[![](ruta.png)](http://127.0.0.1:9966/?z=11&center=43.482072%2C-5.717010&loc=43.394020%2C-5.706718&loc=43.523757%2C-6.047233&hl=en&alt=0)

**EXTRA**

OSRM es una herramienta extremadamente compleja, con muchas funcionalidades muy sencillas de usar. Una de ellas es crear una visualización de la velocidad de ruta de cada tramo de carretera, giro y paso. Además, indica --en rosa-- los tramos de carretera que no están conectados a la red general, por ejemplo por un error al introducir las trazas en OSM.

Para explorar esta vista sólo hay que visitar `http://127.0.0.1:9966/debug/`

[![](debug.png)](http://127.0.0.1:9966/debug/?z=11&center=43.482072%2C-5.717010&loc=43.394020%2C-5.706718&loc=43.523757%2C-6.047233&hl=en&alt=0#16.87/43.39366/-5.80220)


In [1]:
osrm_server="http://0.0.0.0:5000"
#osrm_server="https://router.project-osrm.org" #Use a OSM-based, global, car, osrm server"


In [2]:
#Hacer una llamada sencilla entre 2 puntos

import requests  #http framework to make Mapbox API requests for routes
import json # handle response as json
import datetime # save timestamp

url=osrm_server+"/route/v1/driving/"
comma="%2C"
sep="%3B"

origin=[43.394020,-5.706718]
destination=[43.523757,-6.047233]
fullurl=url+str(origin[1])+','+str(origin[0])+";"+str(destination[1])+','+str(destination[0])
response = requests.get(fullurl) #do the request
response.raise_for_status() # ensure we notice bad responses
print, fullurl
# http://localhost:5000/route/v1/driving/-17.418823,14.721761;-13.95813,13.480448'
print("Entre La Corrada y Noreña hay %2.1f km y ~%2.0f minutos en coche"%
      (json.loads(response.text)['routes'][0]['distance']/1000.,
       json.loads(response.text)['routes'][0]['duration']/60.))
#Entre La Corrada y Noreña hay 48.8 km y ~37 minutos en coche


Entre La Corrada y Noreña hay 48.8 km y ~37 minutos en coche


### Descargar todas los municipios de Asturias, y su población

El [Instituto Geográfico Nacional](http://www.ign.es/web/ign/portal) tiene un fichero `BD_Municipios-Entidades` que contiene una base de datos de todas las ["Entidades"](https://es.wikipedia.org/wiki/Entidad_singular_de_poblaci%C3%B3n) (ciudades, pueblos, aldeas, ...). El archivo incluye, entre otros, los campos que necesitamos (nombre, localización geográfica y población). El formato es `.obd` pero se puede transformar en `.geojson` (con LibreOffice para pasarlo a `.csv`, y Qgis para pasarlo a `.geojson`) con el que podemos trabajar más facilmente. Por conveniencia he subido los archivos transformados a este [gist](https://gist.github.com/brunosan/f52b12ccf7dfba0158b81646fcc7531c)

In [8]:
#descargar archivo si no existe
import sys
import os
import urllib

#carpeta
directory='./data/BD_Municipios-Entidades/'
if not os.path.exists(directory):
    os.makedirs(directory)

file='entidades_asturias.geojson'
url='https://gist.githubusercontent.com/brunosan/f52b12ccf7dfba0158b81646fcc7531c/raw/e20cd38f1d6cf1e803d27a2447f4e5610f48d9dc/entidades_asturias.geojson'
if not os.path.exists(directory+file):
    print("Descargando fichero desde gist.github")
    urllib.request.urlretrieve (url,directory+file)

## Acceso al aeropuerto

Una vez tenemos el enrutador, podemos calcular la curva de acceso de la población asturiana al aeropuerto, o a Somiedo

In [20]:
import geopandas as gpd

fname = "./data/BD_Municipios-Entidades/entidades_asturias.geojson"

df = gpd.read_file(fname)

df

,CODIGOINE,NOMBRE,COD_PROV,PROVINCIA,TIPO,POBLACION,INEMUNI,HOJA_MTN25,lon,lat,orlatlon,ALTITUD,oraltitud,sup,dis,geometry
0,33001000000,Allande,33,Asturias,Municipio,1742,33001,0050-2,-6.608383,43.272294,Mapa,524,MDT5,FALSO,FALSO,POINT (-6.60838301 43.27229373)
1,33001010000,Berducedo,33,Asturias,Entidad colectiva,134,33001,0050-3,-6.767586,43.232347,Mapa,899,MDT5,FALSO,FALSO,POINT (-6.76758642 43.23234691)
2,33001010100,Baldedo,33,Asturias,Entidad singular,4,33001,0050-3,-6.767333,43.247713,Mapa,856,MDT5,FALSO,FALSO,POINT (-6.76733326 43.24771251)
3,33001010199,Baldedo,33,Asturias,Diseminado,4,33001,0050-3,-6.767333,43.247713,Mapa,856,MDT5,FALSO,FALSO,POINT (-6.76733326 43.24771251)
4,33001010200,Berducedo,33,Asturias,Entidad singular,97,33001,0050-3,-6.767586,43.232347,Mapa,899,MDT5,FALSO,FALSO,POINT (-6.76758642 43.23234691)
5,33001010201,Berducedo,33,Asturias,Otras entidades,97,33001,0050-3,-6.767586,43.232347,Mapa,899,MDT5,FALSO,FALSO,POINT (-6.76758642 43.23234691)
6,33001010300,Las Caba?as,33,Asturias,Entidad singular,0,33001,0050-3,-6.750688,43.234087,Mapa,709,MDT5,FALSO,FALSO,POINT (-6.75068824 43.23408669)
7,33001010399,Las Caba?as,33,Asturias,Diseminado,0,33001,0050-3,-6.750688,43.234087,Mapa,709,MDT5,FALSO,FALSO,POINT (-6.75068824 43.23408669)
8,33001010400,Castello,33,Asturias,Entidad singular,1,33001,0050-1,-6.761936,43.252136,Mapa,818,MDT5,FALSO,FALSO,POINT (-6.76193622 43.25213619)
9,33001010499,Castello,33,Asturias,Diseminado,1,33001,0050-1,-6.761936,43.252136,Mapa,818,MDT5,FALSO,FALSO,POINT (-6.76193622 43.25213619)


In [15]:
#Si sumamos toda la población, vemos que hay algo raro, Asturias debería tener ~1 millón
df['POBLACION'].sum()

4101996

In [24]:
#Vemos que las "entidades nos son exclusivas"
df[df['NOMBRE']=='Oviedo']

,CODIGOINE,NOMBRE,COD_PROV,PROVINCIA,TIPO,POBLACION,INEMUNI,HOJA_MTN25,lon,lat,orlatlon,ALTITUD,oraltitud,sup,dis,geometry
9004,33044000000,Oviedo,33,Asturias,Municipio,220301,33044,0029-3,-5.843722,43.362322,Mapa,231,MDT5,FALSO,FALSO,POINT (-5.84372206 43.36232165)
9357,33044190000,Oviedo,33,Asturias,Entidad colectiva,205255,33044,0029-3,-5.843722,43.362322,Mapa,231,MDT5,FALSO,FALSO,POINT (-5.84372206 43.36232165)
9358,33044190100,Oviedo,33,Asturias,Entidad singular,186491,33044,0029-3,-5.843722,43.362322,Mapa,231,MDT5,FALSO,FALSO,POINT (-5.84372206 43.36232165)
9359,33044190101,Oviedo,33,Asturias,Capital de municipio,186491,33044,0029-3,-5.843722,43.362322,Mapa,231,MDT5,FALSO,FALSO,POINT (-5.84372206 43.36232165)
9360,33044190199,Oviedo,33,Asturias,Diseminado,0,33044,0029-3,-5.843722,43.362322,Mapa,231,MDT5,FALSO,FALSO,POINT (-5.84372206 43.36232165)


In [30]:
#Entendemos que 'Entidad singular' es lo más próximo al dato que queremos
lugares=df[df['TIPO']=='Entidad singular']



Asturias tiene 6951 núcleos de población, con un total de 1035830 habitantes


In [62]:
#También vemos que hay lugares que no tienen ubicación geográfica, así que hay que quitarlos
sin_ubicacion=lugares[lugares['lon']==0]
print("Hay %i lugares sin ubicación, que acumulan %i habitantes:\n%s..."%
     (sin_ubicacion['NOMBRE'].count(),sin_ubicacion['POBLACION'].sum(),
     ", ".join(sin_ubicacion['NOMBRE'].values[0:20])))
lugares=lugares[lugares['lon']!=0]

Hay 402 lugares sin ubicación, que acumulan 1614 habitantes:
Rotella, Los Col.laos, Comenencia, Yanalamata, El Taranc?n, La Teyera, Legalid?, El Terceru Legalid?, El Quintu Legalid?, El Val.le, Arnoste, Omeo, Naveo, Les Fureres, L'Omeal, El Pasa?riu, Les Bolgueres, El Pendu la Tabla, Cuevas, Roseco...


In [69]:
sin_poblacion=lugares[lugares['POBLACION']==0]
print("Hay %i lugares sin Población:\n%s..."%
     (sin_poblacion['NOMBRE'].count(),
     ", ".join(sin_ubicacion['NOMBRE'].values[0:30])))
lugares=lugares[lugares['POBLACION']!=0]
lugares=lugares.reset_index(drop=True)

Hay 583 lugares sin Población:
Las Caba?as, El Castro, La Grandera, Trapa, Trellopico, Armenande, La Roza, Bojo, Fresnedo de San Emiliano, Vallinas, Arbeyales, El Sellon, Vallinadosa, Coba, El Provo, Rubieiro, San Mart?n del Valledor, Bustarel, Trabaces, La Pena, La Pruv?a, La Col.l?, Los Corraones, Les Yanes, Pedreo, El Pont?n del Val.le, Cueves, Arteos, Cambrosio, Castiel.lo...


In [70]:
poblacion=lugares['POBLACION'].sum()
num=lugares[lugares['TIPO']=='Entidad singular']['POBLACION'].count()
print("Asturias tiene %i núcleos de población, con un total de %i habitantes" %
     (num,poblacion))

Asturias tiene 5966 núcleos de población, con un total de 1034216 habitantes
